In [ ]:
!pip install transformers datasets

import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
import numpy as np
from transformers import TrainerCallback

In [ ]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast

# kogpt 모델 로드
model_name = 'skt/kogpt2-base-v2' 
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, bos_token='</s>', eos_token='</s>', unk_token='<unk>',
    pad_token='</s>')
model = GPT2LMHeadModel.from_pretrained(model_name)


In [ ]:
# 샘플 텍스트 데이터
texts = [
    "학사제도는 대학의 학사 운영에 대한 제도입니다.",
    "수강 신청은 매 학기 시작 전 진행됩니다.",
    "성적 평가는 절대평가와 상대평가로 나뉩니다.",
    "휴학은 학칙에 따라 신청 가능합니다.",
    "복학은 지정된 기간 내에 신청해야 합니다.",
    "졸업 요건은 전공 이수 학점을 충족해야 합니다.",
    "전과 신청은 일정 기준을 만족해야 합니다.",
    "재수강은 성적 향상을 위해 가능합니다.",
    "학점 인정은 교과목의 성격에 따라 결정됩니다.",
    "수업은 온라인과 오프라인으로 진행됩니다.",
    "시험은 중간고사와 기말고사로 구성됩니다.",
    "학점 포기는 정해진 기간 내에 가능합니다.",
    "장학금 신청은 매 학기 초에 진행됩니다.",
    "교환학생 프로그램은 다양한 국가에서 운영됩니다.",
    "학생 상담은 교수님과 예약 후 가능합니다.",
    "수업 자료는 학교 웹사이트에서 확인할 수 있습니다.",
    "동아리 활동은 학업과 병행 가능합니다.",
    "등록금 납부는 지정된 은행을 통해 진행됩니다.",
    "도서관은 평일과 주말 모두 운영됩니다.",
    "캠퍼스 내 시설은 학생들에게 개방되어 있습니다.",
]

# 데이터셋 분리
train_texts = texts[:2]
test_texts = texts[2:]

# 데이터셋 생성
train_dataset = Dataset.from_dict({'text': train_texts})
test_dataset = Dataset.from_dict({'text': test_texts})


In [ ]:
#토크나이징
def tokenize_function(examples):
    model_inputs = tokenizer(examples["text"], padding='max_length', max_length=512, truncation=True)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

In [ ]:
#파인튜닝
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="epoch", 
    save_strategy="epoch",
    logging_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

trainer.train()
